In [1]:
import torch
from torch.utils.data import Dataset

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# !pip install wandb
import wandb
import random
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler
print(torch.__version__)

2.0.1+cu118


In [2]:

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="transformer_timeseries",
)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kirteshpatel98. Use `wandb login --relogin` to force relogin


In [3]:
'''
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/')


df=pd.read_csv('WTH.csv')
# df.drop('Unnamed: 0',axis=1,inplace=True)
df
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\nimport os\nos.chdir('/content/drive/My Drive/')\n\n\ndf=pd.read_csv('WTH.csv')\n# df.drop('Unnamed: 0',axis=1,inplace=True)\ndf\n"

In [4]:
df=pd.read_csv('ETTh1.csv',index_col=None)
# df.drop('Unnamed: 0',axis=1,inplace=True)
df.iloc[:,1:]

,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,5.827,2.009,1.599,0.462,4.203,1.340,30.531000
1,5.693,2.076,1.492,0.426,4.142,1.371,27.787001
2,5.157,1.741,1.279,0.355,3.777,1.218,27.787001
3,5.090,1.942,1.279,0.391,3.807,1.279,25.044001
4,5.358,1.942,1.492,0.462,3.868,1.279,21.948000
...,...,...,...,...,...,...,...
17415,-1.674,3.550,-5.615,2.132,3.472,1.523,10.904000
17416,-5.492,4.287,-9.132,2.274,3.533,1.675,11.044000
17417,2.813,3.818,-0.817,2.097,3.716,1.523,10.271000
17418,9.243,3.818,5.472,2.097,3.655,1.432,9.778000


In [5]:
scaler = StandardScaler()
df.iloc[:,1:] = scaler.fit_transform(df.iloc[:,1:])
df

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 00:00:00,-0.219049,-0.114207,-0.395683,-0.231903,0.976355,0.805738,2.008513
1,2016-07-01 01:00:00,-0.238009,-0.081400,-0.411356,-0.251800,0.923970,0.857445,1.688203
2,2016-07-01 02:00:00,-0.313849,-0.245432,-0.442557,-0.291043,0.610524,0.602247,1.688203
3,2016-07-01 03:00:00,-0.323329,-0.147013,-0.442557,-0.271146,0.636286,0.703993,1.368010
4,2016-07-01 04:00:00,-0.285409,-0.147013,-0.411356,-0.231903,0.688671,0.703993,1.006610
...,...,...,...,...,...,...,...,...
17415,2018-06-26 15:00:00,-1.280380,0.640341,-1.452403,0.691136,0.348602,1.110975,-0.282568
17416,2018-06-26 16:00:00,-1.820597,1.001211,-1.967580,0.769622,0.400987,1.364505,-0.266225
17417,2018-06-26 17:00:00,-0.645506,0.771566,-0.749583,0.671791,0.558139,1.110975,-0.356458
17418,2018-06-26 18:00:00,0.264287,0.771566,0.171642,0.671791,0.505755,0.959190,-0.414007


In [6]:
class weather_data(torch.utils.data.Dataset):
    def __init__(self, df, mode="train", seq_len=336, pred_len=96,num_feat=7):
        super().__init__()
        self.num_feat=num_feat
        self.df = df.iloc[:,1:num_feat+1]
        # time_stamp = df.iloc[:,0]

        assert mode in ['train', 'test', 'val']
        type_map = {'train': 0, 'val': 1, 'test': 2}
        self.set_type = type_map[mode]

        self.seq_len = seq_len
        self.pred_len = pred_len
        dataset_len=len(df)

        border1s = [0, int(round(0.7*dataset_len,0)) - self.seq_len, int(round(0.7*dataset_len,0)) + int(round(0.1*dataset_len,0)) - self.seq_len]
        border2s = [int(round(0.7*dataset_len,0)), int(round(0.7*dataset_len,0)) + int(round(0.1*dataset_len,0)), int(round(0.7*dataset_len,0)) + int(round(0.1*dataset_len,0)) + int(round(0.2*dataset_len,0))]
        border1 = border1s[self.set_type]
        border2 = border2s[self.set_type]




        self.df = self.df.to_numpy(dtype=np.float32)
        # time_stamp = time_stamp.to_numpy()

        self.data_x = self.df[border1: border2, :]
        self.data_y = self.df[border1: border2, :]

        # self.data_stamp = time_stamp[border1: border2]

    def __getitem__(self, index):
        s_begin = index
        s_end = s_begin + self.seq_len
        r_begin = s_end
        r_end = r_begin + self.pred_len

        seq_x = self.data_x[s_begin:s_end]
        seq_y = self.data_y[r_begin:r_end]
        return seq_x, seq_y


    def __len__(self):
        return len(self.data_x) - self.seq_len - self.pred_len + 1

In [7]:
from torch.utils.data import DataLoader

train_dataset = weather_data(df=df)
valid_dataset = weather_data(df=df,mode="val")
test_dataset = weather_data(df=df,mode="test")

print(len(train_dataset),len(valid_dataset),len(test_dataset))

11763 1647 3389


In [8]:
train_dataset[1][1].shape

(96, 7)

In [9]:
import os
current_directory = os.getcwd()
os.chdir(current_directory+r"\PatchTST\PatchTST_supervised")
from PatchTST.PatchTST_supervised.models.PatchTST import Model
os.chdir(current_directory)
Model

PatchTST.PatchTST_supervised.models.PatchTST.Model

In [10]:
class Configs:
    def __init__(self):
        self.enc_in = 7
        self.seq_len = 336
        self.pred_len = 96
        self.e_layers = 3
        self.n_heads = 4
        self.d_model = 16
        self.d_ff = 128
        self.dropout = 0.3
        self.fc_dropout = 0.
        self.head_dropout = 0.
        self.individual = True
        self.patch_len = 24
        self.stride = 2
        self.padding_patch = True
        self.revin = True
        self.affine = False
        self.subtract_last = False
        self.decomposition = False
        self.kernel_size = 25
        
my_configs = Configs()
model=Model(configs=my_configs)

In [11]:
dat=torch.from_numpy(train_dataset[0][0])
dat=dat.unsqueeze(dim=0)
dat.shape
model(dat).shape

torch.Size([1, 96, 7])

In [12]:
import gc
torch.cuda.empty_cache()
gc.collect()

378

In [13]:
class model_run:
    def __init__(self,model=model):
        self.patchtst_model = model
        
    def model_architecture(self):
        n=0
        for x in self.patchtst_model.state_dict():
            n=n+1
            print(x)
        print("layers= ",n)

    def load_datasets(self,train_dataset=train_dataset,valid_dataset=valid_dataset,test_dataset=test_dataset):
        self.train_dataset=train_dataset
        self.valid_dataset=valid_dataset
        self.test_dataset=test_dataset



    def model_hyperparameters(self,batch_size=128,lr=0.00001,epochs=500,cuda=True,Dataloader=DataLoader):
        self.batch_size = batch_size
        self.lr=lr
        self.epochs=epochs

        self.train_dataloader = DataLoader(self.train_dataset, batch_size=batch_size, shuffle=True)
        self.test_dataloader = DataLoader(self.test_dataset, batch_size=1, shuffle=False)
        self.valid_dataloader = DataLoader(self.valid_dataset, batch_size=batch_size, shuffle=True)


        if cuda:
            self.patchtst_model=self.patchtst_model.to("cuda")

        self.optimizer = torch.optim.Adam(self.patchtst_model.parameters(), lr=self.lr)
        self.loss = torch.nn.MSELoss()
        self.loss_mae=torch.nn.L1Loss()

    def model_execute(self,):
        self.load_datasets()
        self.model_hyperparameters()

        for epoch in range(self.epochs):
            iter_count = 0
            total_loss = 0
            train_steps=0
            total_train_mae_loss=0

            for train_x, train_y in tqdm(self.train_dataloader):
                train_x = train_x.to("cuda")
                train_y = train_y.to("cuda")
                
                self.i=train_x
                pred_y = self.patchtst_model(train_x)
                # print(train_x.shape)
                # print(train_y.shape)
                # print(pred_y.shape)
                train_mae_loss=self.loss_mae(pred_y, train_y)

                loss_t = self.loss(pred_y, train_y)
                self.optimizer.zero_grad()
                loss_t.backward()
                self.optimizer.step()
                total_loss += loss_t.item()
                total_train_mae_loss += train_mae_loss.item()
                iter_count += 1
                train_steps += 1

            valid_iter_count = 0
            valid_total_loss = 0
            valid_total_mae = 0
            with torch.no_grad():
                for valid_x, valid_y in self.valid_dataloader:
                    valid_x = valid_x.to("cuda")
                    valid_y = valid_y.to("cuda")
                    pred_y = self.patchtst_model(valid_x)
                    loss_v = self.loss(pred_y, valid_y)
                    valid_loss_mae=self.loss_mae(pred_y, valid_y)

                    valid_total_loss += loss_v.item()
                    valid_iter_count += 1

                    valid_total_mae+=valid_loss_mae.item()

                total_loss /= iter_count
                total_train_mae_loss /= iter_count

                valid_total_loss /= valid_iter_count
                valid_total_mae /= valid_iter_count
                wandb.log({"MSE Train": total_loss, "MSE Test": valid_total_loss,"MAE Train": total_train_mae_loss,"MAE Test": valid_total_mae})

                print("epoch: {} MSE loss: {:.4f} MSE valid loss: {:.4f}".format(epoch, total_loss, valid_total_loss))
                print("          MAE loss: {:.4f} MAE valid loss: {:.4f}".format(total_train_mae_loss, valid_total_mae))

    def save_model(self,name):
        torch.save(self.patchtst_model.state_dict(), name+'.pth')


    def test_plots(self,ind=3,df_columns=None,scaler=None,column=None):
        test_total_loss = 0
        test_total_loss_mae=0
        test_iter_count = 0
        test_pred_plot=[]
        test_y_plot=[]
        
        df_y_final=pd.DataFrame(columns=df_columns)
        df_pred_final=pd.DataFrame(columns=df_columns)
        
        for test_x, test_y in self.test_dataloader:
            test_x = test_x.to("cuda")
            test_y = test_y.to("cuda")
            test_pred=self.patchtst_model(test_x.to("cuda"))
            # print(test_pred.shape)
            # print(test_y.shape)
            loss_test=self.loss(test_pred, test_y)
            test_total_loss += loss_test.item()
            
            loss_test_mae=self.loss_mae(test_pred, test_y)
            test_total_loss_mae+=loss_test_mae.item()
            
            test_iter_count += 1
            
            
            df_y=pd.DataFrame(test_y[0].to("cpu").numpy(),columns=df_columns)
            df_pred=pd.DataFrame(test_pred[0].to("cpu").detach().numpy(),columns=df_columns)
            
            df_y_final = pd.concat([df_y_final, df_y], axis=0)
            df_pred_final = pd.concat([df_pred_final, df_pred], axis=0)
        df_pred_final=scaler.inverse_transform(df_pred_final)   
        df_y_final=scaler.inverse_transform(df_y_final)
        
        df_pred_final=pd.DataFrame(df_pred_final,columns=df_columns)
        df_y_final=pd.DataFrame(df_y_final,columns=df_columns)
        
        
        sns.lineplot(x=list(range(len(df_pred_final)))[:96], y=df_pred_final[column][:96], label="predicted")
        sns.lineplot(x=list(range(len(df_y_final)))[:96], y=df_y_final[column][:96], label="actual")
        plt.legend(title='Type')
        plt.xlabel('Time Step')
        plt.title("Predicted vs Actual")
        plt.show()
        
        
        test_total_loss/= test_iter_count
        test_total_loss_mae/=test_iter_count
        print("MSE test loss: ",test_total_loss)
        print("MAE test loss: ",test_total_loss_mae)
         
'''
            # plotting
            
            for i,j in zip(test_pred[0],test_y[0]):
                pass
            # print(i[ind].item())
                test_pred_plot.append(i[ind].item())
            # print(j[ind].item())
                test_y_plot.append(j[ind].item())
        sns.lineplot(x=list(range(len(test_pred_plot)))[:96], y=test_pred_plot[:96], label="predicted")
        sns.lineplot(x=list(range(len(test_y_plot)))[:96], y=test_y_plot[:96], label="actual")
        plt.legend(title='Lines')
        plt.show()
        test_total_loss/= test_iter_count
        print("MSE test loss: ",test_total_loss)

        '''


'\n            # plotting\n            \n            for i,j in zip(test_pred[0],test_y[0]):\n                pass\n            # print(i[ind].item())\n                test_pred_plot.append(i[ind].item())\n            # print(j[ind].item())\n                test_y_plot.append(j[ind].item())\n        sns.lineplot(x=list(range(len(test_pred_plot)))[:96], y=test_pred_plot[:96], label="predicted")\n        sns.lineplot(x=list(range(len(test_y_plot)))[:96], y=test_y_plot[:96], label="actual")\n        plt.legend(title=\'Lines\')\n        plt.show()\n        test_total_loss/= test_iter_count\n        print("MSE test loss: ",test_total_loss)\n\n        '

In [14]:
x=model_run(model=model)

In [15]:
x.model_execute()

100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.85it/s]


epoch: 0 MSE loss: 0.7117 MSE valid loss: 0.5018
          MAE loss: 0.6037 MAE valid loss: 0.5160


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.89it/s]


epoch: 1 MSE loss: 0.5886 MSE valid loss: 0.4571
          MAE loss: 0.5484 MAE valid loss: 0.4913


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.91it/s]


epoch: 2 MSE loss: 0.5396 MSE valid loss: 0.4358
          MAE loss: 0.5243 MAE valid loss: 0.4772


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.88it/s]


epoch: 3 MSE loss: 0.5124 MSE valid loss: 0.4228
          MAE loss: 0.5100 MAE valid loss: 0.4684


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.87it/s]


epoch: 4 MSE loss: 0.4944 MSE valid loss: 0.4143
          MAE loss: 0.5004 MAE valid loss: 0.4625


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.85it/s]


epoch: 5 MSE loss: 0.4811 MSE valid loss: 0.4075
          MAE loss: 0.4929 MAE valid loss: 0.4577


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.84it/s]


epoch: 6 MSE loss: 0.4704 MSE valid loss: 0.4017
          MAE loss: 0.4867 MAE valid loss: 0.4534


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 7 MSE loss: 0.4614 MSE valid loss: 0.3969
          MAE loss: 0.4814 MAE valid loss: 0.4501


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.84it/s]


epoch: 8 MSE loss: 0.4536 MSE valid loss: 0.3928
          MAE loss: 0.4768 MAE valid loss: 0.4468


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.84it/s]


epoch: 9 MSE loss: 0.4469 MSE valid loss: 0.3895
          MAE loss: 0.4727 MAE valid loss: 0.4442


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.84it/s]


epoch: 10 MSE loss: 0.4410 MSE valid loss: 0.3858
          MAE loss: 0.4693 MAE valid loss: 0.4415


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.84it/s]


epoch: 11 MSE loss: 0.4353 MSE valid loss: 0.3827
          MAE loss: 0.4656 MAE valid loss: 0.4391


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.84it/s]


epoch: 12 MSE loss: 0.4311 MSE valid loss: 0.3802
          MAE loss: 0.4631 MAE valid loss: 0.4370


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 13 MSE loss: 0.4266 MSE valid loss: 0.3778
          MAE loss: 0.4604 MAE valid loss: 0.4351


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 14 MSE loss: 0.4225 MSE valid loss: 0.3753
          MAE loss: 0.4577 MAE valid loss: 0.4331


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.82it/s]


epoch: 15 MSE loss: 0.4191 MSE valid loss: 0.3738
          MAE loss: 0.4556 MAE valid loss: 0.4317


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 16 MSE loss: 0.4160 MSE valid loss: 0.3712
          MAE loss: 0.4535 MAE valid loss: 0.4296


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 17 MSE loss: 0.4131 MSE valid loss: 0.3692
          MAE loss: 0.4518 MAE valid loss: 0.4278


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.82it/s]


epoch: 18 MSE loss: 0.4103 MSE valid loss: 0.3673
          MAE loss: 0.4498 MAE valid loss: 0.4266


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.82it/s]


epoch: 19 MSE loss: 0.4079 MSE valid loss: 0.3657
          MAE loss: 0.4485 MAE valid loss: 0.4253


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.81it/s]


epoch: 20 MSE loss: 0.4049 MSE valid loss: 0.3653
          MAE loss: 0.4468 MAE valid loss: 0.4245


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.82it/s]


epoch: 21 MSE loss: 0.4029 MSE valid loss: 0.3635
          MAE loss: 0.4455 MAE valid loss: 0.4232


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.82it/s]


epoch: 22 MSE loss: 0.4010 MSE valid loss: 0.3627
          MAE loss: 0.4442 MAE valid loss: 0.4223


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 23 MSE loss: 0.3989 MSE valid loss: 0.3607
          MAE loss: 0.4429 MAE valid loss: 0.4210


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 24 MSE loss: 0.3971 MSE valid loss: 0.3603
          MAE loss: 0.4417 MAE valid loss: 0.4204


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 25 MSE loss: 0.3954 MSE valid loss: 0.3592
          MAE loss: 0.4407 MAE valid loss: 0.4193


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 26 MSE loss: 0.3937 MSE valid loss: 0.3577
          MAE loss: 0.4397 MAE valid loss: 0.4182


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 27 MSE loss: 0.3920 MSE valid loss: 0.3571
          MAE loss: 0.4386 MAE valid loss: 0.4173


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.82it/s]


epoch: 28 MSE loss: 0.3907 MSE valid loss: 0.3556
          MAE loss: 0.4378 MAE valid loss: 0.4165


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.82it/s]


epoch: 29 MSE loss: 0.3893 MSE valid loss: 0.3556
          MAE loss: 0.4369 MAE valid loss: 0.4161


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.82it/s]


epoch: 30 MSE loss: 0.3878 MSE valid loss: 0.3550
          MAE loss: 0.4358 MAE valid loss: 0.4155


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 31 MSE loss: 0.3866 MSE valid loss: 0.3545
          MAE loss: 0.4353 MAE valid loss: 0.4151


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 32 MSE loss: 0.3856 MSE valid loss: 0.3537
          MAE loss: 0.4346 MAE valid loss: 0.4140


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 33 MSE loss: 0.3844 MSE valid loss: 0.3532
          MAE loss: 0.4338 MAE valid loss: 0.4134


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.82it/s]


epoch: 34 MSE loss: 0.3835 MSE valid loss: 0.3531
          MAE loss: 0.4331 MAE valid loss: 0.4133


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.82it/s]


epoch: 35 MSE loss: 0.3826 MSE valid loss: 0.3518
          MAE loss: 0.4326 MAE valid loss: 0.4125


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.82it/s]


epoch: 36 MSE loss: 0.3814 MSE valid loss: 0.3509
          MAE loss: 0.4319 MAE valid loss: 0.4119


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 37 MSE loss: 0.3805 MSE valid loss: 0.3503
          MAE loss: 0.4313 MAE valid loss: 0.4114


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 38 MSE loss: 0.3797 MSE valid loss: 0.3512
          MAE loss: 0.4308 MAE valid loss: 0.4113


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.82it/s]


epoch: 39 MSE loss: 0.3787 MSE valid loss: 0.3494
          MAE loss: 0.4299 MAE valid loss: 0.4103


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 40 MSE loss: 0.3781 MSE valid loss: 0.3500
          MAE loss: 0.4297 MAE valid loss: 0.4103


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 41 MSE loss: 0.3773 MSE valid loss: 0.3494
          MAE loss: 0.4290 MAE valid loss: 0.4100


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.81it/s]


epoch: 42 MSE loss: 0.3766 MSE valid loss: 0.3488
          MAE loss: 0.4284 MAE valid loss: 0.4094


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.81it/s]


epoch: 43 MSE loss: 0.3756 MSE valid loss: 0.3485
          MAE loss: 0.4283 MAE valid loss: 0.4090


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.82it/s]


epoch: 44 MSE loss: 0.3746 MSE valid loss: 0.3481
          MAE loss: 0.4274 MAE valid loss: 0.4086


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 45 MSE loss: 0.3738 MSE valid loss: 0.3474
          MAE loss: 0.4270 MAE valid loss: 0.4081


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 46 MSE loss: 0.3732 MSE valid loss: 0.3476
          MAE loss: 0.4265 MAE valid loss: 0.4080


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.83it/s]


epoch: 47 MSE loss: 0.3727 MSE valid loss: 0.3473
          MAE loss: 0.4262 MAE valid loss: 0.4077


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.82it/s]


epoch: 48 MSE loss: 0.3721 MSE valid loss: 0.3465
          MAE loss: 0.4257 MAE valid loss: 0.4071


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.82it/s]


epoch: 49 MSE loss: 0.3716 MSE valid loss: 0.3469
          MAE loss: 0.4252 MAE valid loss: 0.4070


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.81it/s]


epoch: 50 MSE loss: 0.3706 MSE valid loss: 0.3464
          MAE loss: 0.4249 MAE valid loss: 0.4068


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.81it/s]


epoch: 51 MSE loss: 0.3702 MSE valid loss: 0.3461
          MAE loss: 0.4244 MAE valid loss: 0.4063


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:15<00:00,  5.81it/s]


epoch: 52 MSE loss: 0.3694 MSE valid loss: 0.3459
          MAE loss: 0.4244 MAE valid loss: 0.4062


 24%|███████████████████▌                                                              | 22/92 [00:03<00:12,  5.56it/s]


KeyboardInterrupt: 

MSE test loss:  0.4198806680217891


In [16]:
x.test_plots(df_columns=df.columns[1:],scaler=scaler,column='HULL')

In [320]:
'''
dat=torch.from_numpy(train_dataset[0][0])
dat=dat.unsqueeze(dim=0)
dat.shape
model(dat).shape
'''

'\ndat=torch.from_numpy(train_dataset[0][0])\ndat=dat.unsqueeze(dim=0)\ndat.shape\nmodel(dat).shape\n'